In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv("Human_Activity_Recognition_Using_Smartphones_Data.csv")
df.head()

In [ ]:
print(df["Activity"].unique())
print("Number of class in Activity is: ", df["Activity"].nunique())

In [ ]:
print(df.dtypes)

In [ ]:
print(df.iloc[:, :-1].min().value_counts())
print(df.iloc[:, :-1].max().value_counts())
df['Activity'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df["Activity"].value_counts().plot(kind='bar')

plt.xlabel("Activities")
plt.ylabel("Number of Samples")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

df["Activity_Labels"] = LabelEncoder().fit_transform(df["Activity"])

print(df["Activity_Labels"].unique())
print(df["Activity_Labels"].value_counts())

In [ ]:
feature_cols = df.columns[:-2]
print(feature_cols)
corr_values = df[feature_cols].corr()

tril_index = np.tril_indices_from(corr_values)

for coord in zip(*tril_index):
    corr_values.iloc[coord[0], coord[1]] = np.nan

corr_values = (corr_values
               .stack()
               .to_frame()
               .reset_index()
               .rename(columns={
                   'level_0':'feature1',
                   'level_1':'feature2',
                   0:'correlation'
               })
)

corr_values['abs_correlation'] = corr_values.correlation.abs()

In [ ]:
import seaborn as sns

sns.set_context("talk")
sns.set_style("white")

ax = corr_values.abs_correlation.hist(bins=50, figsize=(12, 8))
ax.set(xlabel="Absolute Correlation", ylabel="Frequency")
plt.show()

In [ ]:
print(corr_values.head())

In [ ]:
filtered = corr_values.sort_values("correlation", ascending=False).query("abs_correlation>0.8")

In [ ]:
unique_features = pd.unique(filtered[['feature1', 'feature2']].values.ravel())
print("Số lượng feature xuất hiện trong các cặp tương quan > 0.8:", len(unique_features))
print("Các feature đó là:", unique_features)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

x = df[feature_cols].dropna()
y = df.loc[x.index, 'Activity_Labels']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


lr_l1 = LogisticRegressionCV(Cs=10, cv=4, penalty='l1', solver='liblinear').fit(x_train, y_train)

In [ ]:
lr_l2 = LogisticRegressionCV(Cs=10, cv=4, penalty='l2', solver='liblinear').fit(x_train, y_train)